# 4. BeautifulSoup 활용
* 멜론사이트 크롤링하기
* 노래제목, 가수명, 앨범명, 좋아요개수 추출

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.melon.com/chart/index.htm'

headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}
res = requests.get(url, headers = headers)
res

<Response [200]>

In [3]:
res.text[:300]

'<!DOCTYPE html>\r\n<html lang="ko">\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<head>\r\n\t\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\r\n\t<meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\r\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\r\n\t\r\n\r\n\t\r\n\r\n\t\r\n\r\n\t<title>멜론차트>TOP100>멜론</title>\r\n\t<meta name="keywor'

In [4]:
soup = BeautifulSoup(res.text)

# 노래 제목 추출

In [5]:
# 1위부터 100위 까지 음악 제목
title = []
for i in range(0, 100):
    soup_title = soup.select('.rank01 a')[i].get_text() # [n] 이때, n은 (음악 순위) - 1
    print(soup_title)
    title.append(soup_title)

# 가수 이름
songer_name = []
songs = soup.select('.rank02')
for song in songs:
    songer_name.append(song.find('a').get_text())

songs = soup.select('.ellipsis.rank03')
album = []
for song in songs:
    album.append(song.find('a').get_text())

신호등
STAY
바라만 본다
Next Level
낙하 (with 아이유)
Queendom
Permission to Dance
Weekend
OHAYO MY NIGHT
Butter
DUMB DUMB
좋아좋아
헤픈 우연
Peaches (Feat. Daniel Caesar & Giveon)
Dynamite
Bad Habits
비와 당신
이제 나만 믿어요
Dun Dun Dance
롤린 (Rollin')
고백
라일락
다정히 내 이름을 부르면
비가 오는 날엔 (2021)
Celebrity
치맛바람 (Chi Mat Ba Ram)
Alcohol-Free
안녕 (Hello)
하루만 더
ASAP
사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
Savage Love (Laxed - Siren Beat) (BTS Remix)
별빛 같은 나의 사랑아
작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)
여전히 아름다운지
가을 우체국 앞에서
운전만해 (We Ride)
HERO
그대라는 사치
봄날
나를 아는 사람
내 손을 잡아
밝게 빛나는 별이 되어 비춰줄게
다시 사랑한다면 (김필 Ver.)
Life Goes On
Off My Face
상상더하기
멜로디
Timeless
밤하늘의 별을(2020)
Lovesick Girls
끝사랑
Bk Love
어떻게 이별까지 사랑하겠어, 널 사랑하는 거지
아로하
잊었니
추억은 만남보다 이별에 남아
어푸 (Ah puh)
비 오는 날 듣기 좋은 노래 (Feat. Colde)
Blueming
상상더하기
Dolphin
미워요
잠이 오질 않네요
에잇(Prod.&Feat. SUGA of BTS)
사랑의 아픔 딛고
슈퍼스타
계단말고 엘리베이터
이젠 잊기로 해요 (Drama Ver.)
그날에 나는 맘이 편했을까
취기를 빌려 (취향저격 그녀 X 산들)
2002
담아 갈게
고백
모든 날, 모든 순간 (Every day, Every Moment)
LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지)
라라라

# 노래 좋아요 수

In [6]:
# 노래 좋아요 수
params = ''
for i in soup.select('.button_etc.like'):
    params += i['data-song-no'] + ','
url = 'https://www.melon.com/commonlike/getSongLike.json?contsIds=' + params[:-1]

headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

res = requests.get(url, headers = headers)
res

<Response [200]>

In [7]:
like = []
l = res.json()['contsLike']

for i in l:
    like.append(i['SUMMCNT'])
# like

In [8]:
# 제목, 가수, 앨범, 좋아요로 dataframe 생성
df_data = {
    'title' : title,
    'singer' : songer_name,
    'album' : album,
    'like' : like,
}
df = pd.DataFrame(data=df_data)
df

,title,singer,album,like
0,신호등,이무진,신호등,190051
1,STAY,The Kid LAROI,Stay,90841
2,바라만 본다,MSG워너비(M.O.M),MSG워너비 1집,147876
3,Next Level,aespa,Next Level,195063
4,낙하 (with 아이유),AKMU (악뮤),NEXT EPISODE,104256
...,...,...,...,...
95,ONLY,이하이,ONLY,10523
96,사실 나는 (Feat.전건호),경서예지,사실 나는,80945
97,On The Ground,로제 (ROSÉ),R,139235
98,나는 너 좋아,장범준,슬기로운 의사생활 시즌2 OST Part 3,19247
